In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import main_module as md

# figure fonts configuration
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import set_config

from lifelines import CoxPHFitter
# import the score function
%run -i ../examples/concordance_index.ipynb

import xgboost as xgb
from catboost import CatBoostRegressor, Pool

from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv

In [ ]:
# import data
from sklearn.model_selection import train_test_split
df_test= pd.read_csv("../data/test_validation_set.csv")
df_train = pd.read_csv("../data/train_set.csv")

## Data preprocessing pipelines
In this section, we create pipelines for preprocessing the data. The main goal here is to investigate if data imputation improves the performance.

In [ ]:
# Naive preprocessor
# replace missing categorical variables by 'missing', replace missing numerical values by -1
class NaiveDataTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = None
    
    def transform(self, X, y=None):
        X_transform = X.copy(deep = True)
        cat_cols = X_transform.select_dtypes(include = 'O').columns
        num_cols = X_transform.select_dtypes(exclude = 'O').columns
        X_transform[cat_cols] = X_transform[cat_cols].fillna("missing")
        X_transform[num_cols] = X_transform[num_cols].fillna(-1.0)
        return X_transform

    def fit(self, X, y=None):
        self.columns = X.columns
        return self 
    
    def get_feature_names_out(self, input_features = None):
        return self.columns

cat_cols = df_train.select_dtypes(include='O').columns
num_cols = df_train.select_dtypes(exclude='O').columns.drop(["ID", 'year_hct','efs', 'efs_time'])
other_cols = df_train.columns.drop(["ID", 'year_hct','efs', 'efs_time'])
# set_config(transform_output="pandas")
preproc_naive = Pipeline(
    steps = [('preprocessing',
                ColumnTransformer([('naive_missing', NaiveDataTransformer(), other_cols),
                                ('ID_year_dropper', 'drop', ["ID", 'year_hct'])],
                                    sparse_threshold=0,
                                    remainder='passthrough',
                                    verbose_feature_names_out=False,
                                    force_int_remainder_cols=False
                                ).set_output(transform="pandas")
            ),
            ('naive_one_hot_encode',
                ColumnTransformer([('one_hot', OneHotEncoder(drop='first',
                                                             min_frequency = 0.001,
                                                             handle_unknown='ignore',
                                                             sparse_output= False), cat_cols)],
                                    sparse_threshold=0,
                                    remainder='passthrough',
                                    verbose_feature_names_out=False,
                                    force_int_remainder_cols=False
                                ).set_output(transform="pandas")
            )
    ]
)

In [ ]:
# Preprocessing based on KNN imputation 
class MissingValueTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, null_list = ["Missing Disease Status", "Missing disease status"]):
        self.null_list = null_list
        self.columns = None
    
    def transform(self, X, y=None):
        X_transform = X.copy(deep = True)
        X_transform.replace(self.null_list, np.nan, inplace = True)
        return X_transform

    def fit(self, X, y=None):
        self.columns = X.columns
        return self 
    
    def get_feature_names_out(self, input_features = None):
        return self.columns

cat_cols = df_train.select_dtypes(include='O').columns
preproc_sd = Pipeline(
    [   
        ('preprocessing',
                ColumnTransformer([
                                    ('cat_missing', MissingValueTransformer(), cat_cols),
                                    ('ID_year_dropper', 'drop', ["ID", 'year_hct'])],
                                    sparse_threshold=0,
                                    remainder='passthrough',
                                    verbose_feature_names_out=False,
                                    force_int_remainder_cols=False
                                ).set_output(transform="pandas")
        ),
        (
            "encode_and_scale",
            ColumnTransformer(
                [
                    ('one_hot', 
                    OneHotEncoder(drop='first',
                                    min_frequency = 0.001,
                                    handle_unknown='ignore',
                                    sparse_output= False
                    ), 
                    cat_cols
                    ),
                    ('scale', StandardScaler(), ['donor_age', 'age_at_hct', 'karnofsky_score'])
                ],
                sparse_threshold=0,
                remainder='passthrough',
                verbose_feature_names_out=False,
                force_int_remainder_cols=False
            ).set_output(transform="pandas")
        ),
        (
            "impute",
            KNNImputer().set_output(transform = "pandas")
        ),
    ]
)

## Modeling Methods

In this section, we implement the actual modeling methods including 
* CoxPH model
* XGboost AFT
* Catboost AFT
* Survival Random Foreast

In [ ]:
# cox propotional harzard model
def cph_model(X_train_preproc, y_train, X_test_preproc):

    train_preproc = pd.concat([X_train_preproc, y_train], axis=1)
    cph = CoxPHFitter()
    cph.fit(train_preproc, duration_col='efs_time', event_col='efs')
    preds = cph.predict_partial_hazard(X_test_preproc)
    
    return preds

In [ ]:
## XGboost
params = {'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'aft_loss_distribution': 'normal',
         'aft_loss_distribution_scale': 0.80,
          'tree_method': 'hist', 'learning_rate': 0.05, 'max_depth': 6}
   
def xgb_aft_model(X_train_preproc, y_train, X_test_preproc, params = params):

    # remove special character
    X_train_preproc.columns = X_train_preproc.columns.str.replace('<','')
    X_test_preproc.columns = X_test_preproc.columns.str.replace('<','')

    y_lower_bound = y_train['efs_time'].copy(deep = True)
    y_upper_bound = y_train['efs_time'].copy(deep = True)
    y_upper_bound[y_train['efs'] == 0.0] = +np.inf

    dtrain = xgb.DMatrix(X_train_preproc)
    dtrain.set_float_info('label_lower_bound', y_lower_bound)
    dtrain.set_float_info('label_upper_bound', y_upper_bound)

    bst = xgb.train(params, dtrain, num_boost_round=500, evals=[(dtrain, 'train')], verbose_eval = 0)
    dtest = xgb.DMatrix(X_test_preproc)
    preds = bst.predict(dtest)

    return -preds

In [ ]:
## Catboost

# catboost does not directly take one-hot encoding
# instead, it requires an explicit declaration of catgorical features

# Here are the modified pipelines for catboost
# The basic idea is to remove the one-hot encoding process

num_cols = df_train.select_dtypes(exclude='O').columns.drop(["ID", 'year_hct','efs', 'efs_time'])
cb_preproc_naive = NaiveDataTransformer()
cb_preproc_sd = Pipeline(
    [   
        ('preprocessing',
                ColumnTransformer([
                                    ('cat_missing', MissingValueTransformer(), cat_cols),
                                    ('ID_year_dropper', 'drop', ["ID", 'year_hct']),
                                    ('scale', StandardScaler(), ['donor_age', 'age_at_hct', 'karnofsky_score'])],
                                    sparse_threshold=0,
                                    remainder='passthrough',
                                    verbose_feature_names_out=False,
                                    force_int_remainder_cols=False
                                ).set_output(transform="pandas")
        ),
        ('impute',
                ColumnTransformer([("num_KNNimpute", KNNImputer(), num_cols),
                                   ("cat_indicate", NaiveDataTransformer(), cat_cols)],
                                    sparse_threshold=0,
                                    remainder='passthrough',
                                    verbose_feature_names_out=False,
                                    force_int_remainder_cols=False
                                ).set_output(transform="pandas")
        )
    ]
)


def cb_aft_model(X_train, y_train, X_test, y_test, pipeline):

    X_train_proc = pipeline.fit_transform(X_train)
    X_test_proc = pipeline.fit_transform(X_test)

    y_lower_train = y_train[['efs_time']].copy(deep = True)
    y_upper_train = y_train[['efs_time']].copy(deep = True)
    # in catboost, infinity is represented by -1
    y_upper_train.iloc[y_train['efs'] == 0.0] = -1

    train_label = np.concatenate((y_lower_train, y_upper_train), axis = 1)
    train_label = pd.DataFrame(train_label, columns = ['y_lower_train', 'y_upper_train'])
    cat_features = list(X_train.select_dtypes(include= 'O').columns)

    y_lower_test = y_test[['efs_time']].copy(deep = True)
    y_upper_test = y_test[['efs_time']].copy(deep = True)
    # in catboost, infinity is represented by -1
    y_upper_test.iloc[y_test['efs'] == 0.0] = -1
    
    test_label = np.concatenate((y_lower_test, y_upper_test), axis = 1)
    test_label = pd.DataFrame(test_label, columns = ['y_lower_test', 'y_upper_test'])

    train_pool = Pool(X_train_proc,label = train_label, cat_features= cat_features)
    test_pool = Pool(X_test_proc,label = test_label, cat_features= cat_features)

    model_normal = CatBoostRegressor(iterations=500,
                                 loss_function='SurvivalAft:dist=Normal',
                                 eval_metric='SurvivalAft',
                                 verbose=0
                                )
    _ = model_normal.fit(train_pool, eval_set=test_pool)
    preds = model_normal.predict(test_pool, prediction_type='Exponent')
    
    return -preds

In [ ]:
#Random survival forest
def rsf_model(X_train_preproc, y_train, X_test_preproc):

    y_train = Surv.from_dataframe("efs", "efs_time", y_train)

    rsf = RandomSurvivalForest(
        n_estimators=30,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        n_jobs=4,
        verbose=1,
        random_state=42
    )
    rsf.fit(X_train_preproc, y_train)
    surv_funcs = rsf.predict_survival_function(X_test_preproc, return_array=False)
    preds = np.array([-np.trapz(fn.y, fn.x) for fn in surv_funcs])
    
    return preds

## Five-fold Cross validation

Here we use the five fold cross validation for getting a baseline results

In [ ]:
# helper functions
# eval evalutes stratified c-index and c-index
def eval(preds, X_test, solution):
    prediction= pd.DataFrame({"ID":X_test["ID"], "prediction":preds})
    sc_score = score(solution.copy(deep=True), prediction.copy(deep=True), "ID")
    c_index = concordance_index(y_test['efs_time'], -preds, y_test['efs'])

    return sc_score, c_index

# file_output export all the information into a csv file
def file_output(filename, sc_indexes):
    sc_mean = sc_indexes.mean()
    output = pd.DataFrame(np.concatenate(
                                        (sc_indexes.to_numpy(), np.expand_dims(sc_mean.to_numpy(), axis = 0)
                                     ), axis = 0), 
                                     index=[0,1,2,3,4, 'mean'],
                                     columns= ['cph', 'xgb_aft', 'cat_aft', 'rsf'])
    output.to_csv(filename, sep= '\t', index= True, header= True)

In [ ]:
preproc_pipline = preproc_naive

n_splits = 5
kfold = KFold(n_splits = n_splits, shuffle = True, random_state = 42)
target_features = ['efs', 'efs_time']

methods_list = ['cph', 'xgb_aft', 'cat_aft', 'rsf']
sc_indexes = -1.0 * np.ones((n_splits, len(methods_list))) 
sc_indexes = pd.DataFrame(data = sc_indexes, columns = methods_list)

for i, (train_idx,test_idx) in enumerate(kfold.split(df_train)):

    X_train = df_train.iloc[train_idx].drop(columns = target_features)
    y_train = df_train.loc[train_idx, target_features]

    X_test = df_train.iloc[test_idx].drop(columns = target_features)
    y_test = df_train.loc[test_idx, target_features]

    preproc_pipline.fit(X_train)
    X_train_preproc = preproc_pipline.transform(X_train)
    X_test_preproc =preproc_pipline.transform(X_test)
    
    preds_cph = cph_model(X_train_preproc, y_train, X_test_preproc)
    preds_xgb = xgb_aft_model(X_train_preproc, y_train, X_test_preproc, params = params)
    preds_cb = cb_aft_model(X_train, y_train, X_test, y_test, cb_preproc_naive)
    preds_rsf = rsf_model(X_train_preproc, y_train, X_test_preproc)

    solution = df_train.iloc[test_idx]
    score_cph, c_index_cph = eval(preds_cph, X_test, solution)
    score_xgb, c_index_xgb = eval(preds_xgb, X_test, solution)
    score_cb, c_index_cb = eval(preds_cb, X_test, solution)
    score_rsf, c_index_rsf = eval(preds_rsf, X_test, solution)

    print(f"stratified c-index for fold {i}: \n \
            SC-index: cph: {score_cph}, xgb_aft: {score_xgb}, cat_aft: {score_cb}, rsf_aft: {score_rsf} \n \
            C_index: cph: {c_index_cph}, xgb_aft: {c_index_xgb}, cat_aft: {c_index_cb}, rsf_aft: {c_index_rsf} \n")
    
    sc_indexes.loc[i, 'cph'] = score_cph
    sc_indexes.loc[i, 'xgb_aft'] = score_xgb
    sc_indexes.loc[i, 'cat_aft'] = score_cb
    sc_indexes.loc[i, 'rsf'] = score_rsf

In [ ]:
print("mean stratified c-index across all 5 folds:")
print(sc_indexes.mean())

file_output('output_naive(baseline)_cv.csv', sc_indexes)

In [ ]:
preproc_pipline = preproc_sd

n_splits = 5
kfold = KFold(n_splits = n_splits, shuffle = True, random_state = 42)
target_features = ['efs', 'efs_time']

methods_list = ['cph', 'xgb_aft', 'cat_aft', 'rsf']
sc_indexes = -1.0 * np.ones((n_splits, len(methods_list))) 
sc_indexes = pd.DataFrame(data = sc_indexes, columns = methods_list)

for i, (train_idx,test_idx) in enumerate(kfold.split(df_train)):

    X_train = df_train.iloc[train_idx].drop(columns = target_features)
    y_train = df_train.loc[train_idx, target_features]

    X_test = df_train.iloc[test_idx].drop(columns = target_features)
    y_test = df_train.loc[test_idx, target_features]

    preproc_pipline.fit(X_train)
    X_train_preproc = preproc_pipline.transform(X_train)
    X_test_preproc =preproc_pipline.transform(X_test)
    
    preds_cph = cph_model(X_train_preproc, y_train, X_test_preproc)
    preds_xgb = xgb_aft_model(X_train_preproc, y_train, X_test_preproc, params = params)
    preds_cb = cb_aft_model(X_train, y_train, X_test, y_test, cb_preproc_sd)
    preds_rsf = rsf_model(X_train_preproc, y_train, X_test_preproc)

    solution = df_train.iloc[test_idx]
    score_cph, c_index_cph = eval(preds_cph, X_test, solution)
    score_xgb, c_index_xgb = eval(preds_xgb, X_test, solution)
    score_cb, c_index_cb = eval(preds_cb, X_test, solution)
    score_rsf, c_index_rsf = eval(preds_rsf, X_test, solution)

    sc_indexes.loc[i, 'cph'] = score_cph
    sc_indexes.loc[i, 'xgb_aft'] = score_xgb
    sc_indexes.loc[i, 'cat_aft'] = score_cb
    sc_indexes.loc[i, 'rsf'] = score_rsf

    print(f"stratified c-index for fold {i}: \n \
            SC-index: cph: {score_cph}, xgb_aft: {score_xgb},  cat_aft: {score_cb}, rsf_aft: {score_rsf} \n \
            C_index: cph: {c_index_cph}, xgb_aft: {c_index_xgb}, cat_aft: {c_index_cb}, rsf_aft: {c_index_rsf}")

In [ ]:
print("Ray pipeline mean stratified c-index across all 5 folds:")
print(sc_indexes.mean())

file_output('output_proc(Ray)_cv.csv', sc_indexes)